In [1]:
import numpy as np
import sys
import matplotlib.pyplot as plt

sys.path.insert(0, './TetrisEnv')
from TetrisBattle.envs.tetris_env import TetrisSingleEnv, TetrisEnv
from TetrisBattle import *
from Model.Feature_Extraction import TetrisActorCriticCnnPolicy
# %pip install -e ./TetrisEnv
# import TetrisBattle

pygame 2.5.2 (SDL 2.28.3, Python 3.10.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
class InputGetter():
    def __init__(self):
        self.input_log = []
        
    def get_input(self):
        while True:
            try:
                if len(self.input_log) == 0:
                    self.input_log = [*input(">")]
                
                inp = self.input_log.pop(0)
                inp = int(inp)
                assert(inp in [0, 1, 2, 3, 4, 5, 6, 7, 9])
                return inp
            except Exception as e:
                print(e)

In [ ]:
env = TetrisSingleEnv(obs_type="grid")
i = 0
inputter = InputGetter()
while True:
    action = inputter.get_input()
    if action == 9:
        env.close()
        break
    state, reward, done, info = env.one_step(action)
    # print("infow", info)
    # print("rewrd",reward)
    env.render()
    # print(env.game_interface.get_obs().shape)
    grid = env.game_interface.get_obs()

In [ ]:
grid[:,:,0]
print(grid.shape)
bool_grid = grid != 0
for i in range(len(grid)):
    for j in range(len(grid[0,:15])):
        num = float(grid[i,j,0])
        # print(f"{bool_grid[i,j,0]}, ", end="")
        print(f"{round(num, 1)}, ", end="")
    print()

In [ ]:
env = TetrisSingleEnv()
i = 0
inputter = InputGetter()
while True:
    action = inputter.get_input()
    if action == 9:
        env.close()
        break
    state, reward, done, info = env.og_step(action)
    print("infow", info)
    print("rewrd",reward)
    env.render()
    print(env.get_obs())

In [ ]:

# env = make('SinglePTetris-v0')
env = TetrisSingleEnv()
ob = env.reset()
dots = np.ones((20, 10))
x = 100
y = 130
ob[x:x+10,y:y+10] = [255,255,255]
plt.imshow(ob, cmap="plasma")


In [3]:
from gym.wrappers import GrayScaleObservation
from preprocessing.EensyWeensy import MakeEensyWeensy
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# from gym import make

naked_env = TetrisSingleEnv(obs_type="grid")

# env = GrayScaleObservation(env, keep_dim=True)
# env = MakeEensyWeensy(env, cut_in_half=True, scale=.25)
env = DummyVecEnv([lambda: naked_env])
env = VecFrameStack(env, 4, channels_order="last")

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
/opt/miniconda3/envs/ECE_113_Capstone_1/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: Y

In [ ]:
ob,_,_,_ = env.step([0])
ob.transpose(3, 0, 1, 2).shape

In [4]:
print(env.action_space)

Discrete(40)


In [4]:
import os 
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.env_checker import check_env
import torch
print(torch.cuda.is_available())
# check_env(TetrisSingleEnv())

False


In [5]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [6]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [7]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [12]:
from Model.Feature_Extraction import TetisFeatureExtractor
feat_ext = TetisFeatureExtractor(naked_env.observation_space)

In [8]:
try:
    del model
except NameError:
    pass
model = PPO(TetrisActorCriticCnnPolicy, env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=1e-2, n_steps=128) 
# TetrisActorCriticCnnPolicy

Using cpu device


RuntimeError: Given groups=1, weight of size [10, 1, 5, 5], expected input[1, 4, 20, 20] to have 1 channels, but got 4 channels instead

In [ ]:
model.learn(total_timesteps=100000, callback=callback)

In [9]:
# model = PPO.load('./train/best_model_20000')
env = TetrisSingleEnv(obs_type="grid")
state = env.reset()
i = 0

while True:
    if (i >= 1000):
        env.close()
        break
    # action, _ = model.predict(state)
    action = env.action_space.sample()
    print(action)
    state, reward, done, info = env.step(action)
    print(state.shape)
    print("reward", reward)
    env.render()
    i += 1

19


IndexError: invalid index to scalar variable.

In [10]:
state.shape

(1, 20, 34, 4)